In [1]:
import pandas as pd
import json
import os

In [6]:
base_output_dir = "/Users/isammohdibrahim/Documents/Dev/running-with-oliver-cromwell/output"
input_file = "/Users/isammohdibrahim/Documents/Dev/running-with-oliver-cromwell/subsample.csv"

#this is uaully a reference to the accession or source of the data
groupname = "crc_sub1" 

ref_version = "b37" # or hg38 
#this is location of the reference files on ARASHI. Don't change unless intending to use the pipeline with own set of references not defined here
ref_dir = f"/home/imibrahim/refs"

# this notebook expects the aligned bam files for tumor and normal samples to be separated
aligned_bam_dir = "/home/imibrahim/crc_sub1/bam"
n_dir = f'{aligned_bam_dir}/{ref_version}-n'
t_dir = f'{aligned_bam_dir}/{ref_version}-t'

send_email_for_all = True
email = "isambmi@hawaii.edu"

#this refers to the container (singularity/docker) used for the pipeline. These containers are located at /home/imibrahim/SIFs
gatk_docker = "arashi-gatk-426"

# read input csv
input_df = pd.read_csv(input_file)
#input must have
# RG name, can be same as sample name
# sample name
# subject_id
# sample type: T/N
# absolute path to fastq1
# absolute path to fastq2
# library_name, can be same as sample name
# platform unit, optional
# sequence date
# sequence platform
# sequence center

In [7]:
input_df

,readgroup,sample_name,subject_id,sample_type,absolute_path_to_fq1,absolute_path_to_fq2,library_name,platform_unit,sequence_date,sequence_platform,sequence_center
0,DCR002_NA,DCR002_NA,DCR002,N,/home/imibrahim/LTS/crc-temp/DCR002_NA/V350150...,/home/imibrahim/LTS/crc-temp/DCR002_NA/V350150...,DCR002_NA,DCR002_NA,2022-05-31T11:52:00,ILLUMINA,BGI
1,DCR002_TA,DCR002_TA,DCR002,N,/home/imibrahim/LTS/crc-temp/DCR002_TA/V350150...,/home/imibrahim/LTS/crc-temp/DCR002_TA/V350150...,DCR002_TA,DCR002_TA,2022-05-31T11:52:00,ILLUMINA,BGI


Make sure to check if the interval file (`interval_loc`) used in the next block matches the data

In [9]:
# switching of references depending on ref_version
if ref_version == "b37":
    interval_loc=f"{ref_dir}/Agilent_SureSelect_Human_All_Exon_V4/S03723314_Regions.converted.bed"

    ref_fasta = f"{ref_dir}/b37/human_g1k_v37_decoy.fasta"
    ref_fai = f"{ref_dir}/b37/human_g1k_v37_decoy.fasta.fai"
    ref_dict = f"{ref_dir}/b37/human_g1k_v37_decoy.dict"
    

    pon = f"{aligned_bam_dir}/pon/{groupname}_pon.vcf"
    pon_idx = f"{aligned_bam_dir}/pon/{groupname}_pon.vcf.idx"

    gnomad = f"{ref_dir}/b37/af-only-gnomad.raw.sites.vcf"
    gnomad_idx = f"{ref_dir}/b37/af-only-gnomad.raw.sites.vcf.idx"
    variants_for_contamination = f"{ref_dir}/b37/small_exac_common_3.vcf"
    variants_for_contamination_idx = f"{ref_dir}/b37/small_exac_common_3.vcf.idx"
    funco_data_source = f"{ref_dir}/b37/funcotator_dataSources.v1.7.20200521s.tar.gz"
elif ref_version == "hg38":
    # only WGS version of interval list on disk for hg38
    interval_loc = f"{ref_dir}/hg38/wgs_calling_regions.hg38.interval_list"

    ref_fasta = f"{ref_dir}/hg38/Homo_sapiens_assembly38.fasta"
    ref_fai = f"{ref_dir}/hg38/Homo_sapiens_assembly38.fasta.fai"
    ref_dict = f"{ref_dir}/hg38/Homo_sapiens_assembly38.dict"

    pon = f"{{aligned_bam_dir}}/pon/1000g_pon.hg38.vcf.gz"
    pon_idx = f"{{aligned_bam_dir}}/pon/1000g_pon.hg38.vcf.gz.tbi"

    gnomad = f"{ref_dir}/hg38/af-only-gnomad.hg38.vcf.gz"
    gnomad_idx = f"{ref_dir}/hg38/af-only-gnomad.hg38.vcf.gz.tbi"
    variants_for_contamination = f"{ref_dir}/hg38/small_exac_common_3.hg38.vcf.gz"
    variants_for_contamination_idx = f"{ref_dir}/hg38/small_exac_common_3.hg38.vcf.gz.tbi"
    funco_data_source = f"{ref_dir}/hg38/funcotator_dataSources.v1.7.20200521s.tar.gz"

# A. SCMA input

In [10]:
scma_output_dir=f'{base_output_dir}/scma'
os.makedirs(scma_output_dir, exist_ok=True)

for index, row in input_df.iterrows():
    i = index + 1
    if send_email_for_all == False and i == input_df.size[0]:
        send_email = True
    else:
        send_email = send_email_for_all

    output = {
            "seqConvMarkAdapt.readgroup_name": row['readgroup'],
            "seqConvMarkAdapt.sample_name": row['sample_name'],
            "seqConvMarkAdapt.fastq_1": row['absolute_path_to_fq1'],
            "seqConvMarkAdapt.fastq_2": row['absolute_path_to_fq2'],
            "seqConvMarkAdapt.library_name": row['library_name'],
            "seqConvMarkAdapt.platform_unit": row['platform_unit'],
            "seqConvMarkAdapt.run_date": row['sequence_date'],
            "seqConvMarkAdapt.platform_name": row['sequence_platform'],
            "seqConvMarkAdapt.sequencing_center": row['sequence_center'],
            "seqConvMarkAdapt.make_fofn": True,
            "seqConvMarkAdapt.send_email": send_email,  
            "seqConvMarkAdapt.email": email,  
        }
    
    with open(f'{scma_output_dir}/{row["readgroup"]}.json', 'w') as outfile:
        json.dump(output, outfile, indent=4)
    output = {}

print(f'{i} input files generated @')
print(f'{scma_output_dir}')


2 input files generated @
/Users/isammohdibrahim/Documents/Dev/running-with-oliver-cromwell/output/scma


# B. Pre-processing
- no input needs to be generated. Pipeline will use output from SCMA.

# C. PoN input

In [ ]:
pon_output_dir = f'{base_output_dir}/m2'
os.makedirs(pon_output_dir, exist_ok=True)

n = input_df.query('sample_type == "N"')
samples_n = n['sample_name'].sort_values().unique().tolist()
n_bams = [f'{n_dir}/{n}.{ref_version}.bam' for n in samples_n]
n_bais = [f'{n_dir}/{n}.{ref_version}.bai' for n in samples_n]

input = {
    "Mutect2_Panel.gatk_docker": gatk_docker,

    "Mutect2_Panel.pon_name": f"{groupname}_pon",
    "Mutect2_Panel.normal_bams": n_bams,
    "Mutect2_Panel.normal_bais": n_bais,


    "Mutect2_Panel.ref_fasta": ref_fasta,
    "Mutect2_Panel.ref_fai": ref_fai,
    "Mutect2_Panel.ref_dict": ref_dict,
    "Mutect2_Panel.scatter_count": 1,
    
    "Mutect2_Panel.gnomad": gnomad,
    "Mutect2_Panel.gnomad_idx": gnomad_idx,

    "Mutect2_Panel.intervals":interval_loc,
    "Mutect2_Panel.email": email,
    "Mutect2_Panel.Mutect2.filter_mem": 2000,
    "Mutect2_Panel.send_email": True if send_email_for_all else False
}

with open(f'{pon_output_dir}/pon.json', 'w') as f:
    json.dump(input, f, indent=4)

print('PoN input generated at:')
print(f'{pon_output_dir}/pon.json')

# D. Mutect2 input

In [ ]:
m2_output_dir = f'{base_output_dir}/m2'

i = 0
os.makedirs(m2_output_dir, exist_ok=True)

# get list of subject ids
subject_ids = input_df["subject_id"].unique()
t = input_df.query('sample_type == "T"')

# parse through list and get tumor and normal sample id for each one
for s in subject_ids:
    i+= 1
    
    # this just attaches a send email job upon completion of the last workflow in the group
    # send_email = True if i == len(subject_ids) else False
    send_email = True
    
    n_sample = n.query(f'subject_id == "{s}" ')["sample_name"].values[0]
    t_sample = t.query(f'subject_id == "{s}" ')["sample_name"].values[0]
    m2_input = {
        
        "Mutect2.normal_reads": f"{n_dir}/{n_sample}.{ref_version}.bam",
        "Mutect2.normal_reads_index": f"{n_dir}/{n_sample}.{ref_version}.bai",
        "Mutect2.tumor_reads": f"{t_dir}/{t_sample}.{ref_version}.bam",
        "Mutect2.tumor_reads_index": f"{t_dir}/{t_sample}.{ref_version}.bai",
        
        "Mutect2.gatk_docker": gatk_docker,
  
        "Mutect2.intervals": interval_loc,
        "Mutect2.scatter_count": 24,
        "Mutect2.m2_extra_args": " -ip 100 ",
        "Mutect2.split_intervals_extra_args": " --subdivision-mode BALANCING_WITHOUT_INTERVAL_SUBDIVISION --dont-mix-contigs --min-contig-size 1000000 ",

        "Mutect2.filter_funcotations": "True",
        "Mutect2.funco_reference_version": ref_version, 
        # funcotator sources for all references are in here
        "Mutect2.funco_data_sources_tar_gz": funco_data_source,

        "Mutect2.ref_fasta": ref_fasta, 
        "Mutect2.ref_fai": ref_fai, 
        "Mutect2.ref_dict": ref_dict, 

        "Mutect2.pon": pon, 
        "Mutect2.pon_idx": pon_idx, 

        "Mutect2.gnomad": gnomad, 
        "Mutect2.gnomad_idx": gnomad_idx, 
        "Mutect2.variants_for_contamination": variants_for_contamination, 
        "Mutect2.variants_for_contamination_idx": variants_for_contamination_idx, 

        "Mutect2.run_funcotator": True,
        "Mutect2.run_orientation_bias_mixture_model_filter": True,
        "Mutect2.send_email": send_email
    }
    with open(f'{m2_output_dir}/{s}.json', 'w') as f:
        json.dump(m2_input, f, indent=4)
print(f'{i} inputs generated @:')
print(f'{m2_output_dir}')